In [153]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import copy
import openpyxl

def ConnectDB():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    return client

def Today():
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    return today

def SaveResult(client, code, date, strategy, data):
    db = client.Result
    collection = db.Recommend
    collection.replace_one({
        "code": code,
        "date": date,
        "strategy": strategy
    }, data, upsert=True)
    
def OpenXlsx(sheet_name):
    file_name = Today().strftime("%Y%m%d.xlsx")
    path = os.path.expanduser("~/Desktop/" + file_name)

    if os.path.exists(path):
        wb = openpyxl.load_workbook(path)
    else:
        wb = openpyxl.Workbook()
        del wb["Sheet"]
    if sheet_name in wb.sheetnames:
        del wb[sheet_name]
    ws = wb.create_sheet(sheet_name)
    return wb, ws

In [154]:
def Strategy_1():
    client = ConnectDB()
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        data = {
            "code": code,
            "name": item["name"],
            "strategy": 1,
            "change_0": changes[0],
            "change_-1": changes[1],
            "change_-2": changes[2],
            
        }
        found.append(data)
        data["date"] = today
        SaveResult(client, code, today, 1, data)

    client.close()
    
    if len(found) > 0:
        df = pandas.DataFrame(found, columns=["code", "name", "change_0", "change_-1", "change_-2"])
    else:
        return None
    
    # Write Excel
    sheet_name = u"策略1_强势股"
    wb, ws = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 18, 18, 18]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    head_alignment = openpyxl.styles.Alignment(horizontal="center", vertical="center")
    head_fill = openpyxl.styles.fills.PatternFill(
        fgColor="D7E4BC",
        fill_type="solid",
        patternType="solid")
    head_font = openpyxl.styles.fonts.Font(
        bold=True,
        sz=14
    )
    for i, v in enumerate(df.columns.values):
        cell = ws.cell(row=1, column=i+1)
        cell.value = v
        cell.alignment = head_alignment
        cell.fill = head_fill
        cell.font = head_font

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            cell.value = v
            if j >= 2:
                cell.number_format = "0.00"

    wb.save(path)
        
    return df

In [34]:
def Strategy_4():
    client = ConnectDB()
    
    today = Today()
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value": value / 1e8,
            "amount": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 4
        SaveResult(client, item["code"], today, 4, data)
        
    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount", "value"]))

    client.close()
    
    return found

In [ ]:
def Strategy_2():
    client = ConnectDB()
    
    today = Today()
    conceptDict = {}
    Threshold = 9.9
    
    for item in client.Common.Stock.find({}):
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        change = (res["close"] / res["last_close"] - 1) * 100
        if change < Threshold:
            continue
    
        for concept in item["concept"]:
            if concept == u"次新股":
                continue
            if concept not in conceptDict:
                conceptDict[concept] = []
            data = {
                "code": item["code"],
                "name": item["name"],
                "change": change
            }
            conceptDict[concept].append(data)
    
    concepts = []
    for k, v in conceptDict.iteritems():
        if len(v) < 3:
            continue
        data = {
            "concept": k,
            "stocks": v,
            "count": len(v),
            "codes": []
        }
        for s in v:
            data["codes"].append(s["code"])
        concepts.append(data)
    
    found = sorted(concepts, key=lambda item: item["count"], reverse=True)
    
    print(pandas.DataFrame(found, columns=["concept", "count", "codes"]))
    
    stockDict = {}
    for k in found:
        for s in k["stocks"]:
            code = s["code"]
            if code not in stockDict:
                stockDict[code] = {
                    "code": code,
                    "name": s["name"],
                    "change": s["change"],
                    "concepts": [],
                    "date": today,
                    "strategy": 2,
                }
            stockDict[code]["concepts"].append(k["concept"])
    for code, v in stockDict.iteritems():
        SaveResult(client, code, today, 2, v)
        
    client.close()

In [ ]:
def FilterSameStrategy():
    client = ConnectDB()
    today = Today()
    
    stocks = {}
    for s in client.Result.Recommend.find({"date": today}):
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": []
            }
        stocks[code]["strategies"].append(s["strategy"])
        
    result = []
    for k, v in stocks.iteritems():
        if len(v["strategies"]) > 1:
            result.append({
                "code": k,
                "name": v["name"],
                "strategies": v["strategies"]
            })
    print pandas.DataFrame(result)

In [ ]:
print("Strategy 1")
Strategy_1()
print("\nStrategy 2")
Strategy_2()
print("\nStrategy 3")
Strategy_3()
print("\nStrategy 4")
Strategy_4()
print("\nSummary")
FilterSameStrategy()

In [ ]:
def Callback():
    client = ConnectDB()
    today = Today()
    minDate = today + datetime.timedelta(days=-10)
    collection = client.Result.Recommend
    cursor = collection.find({"date": {"$gt": minDate}}) \
        .sort("date", pymongo.DESCENDING)

    stocks = {}
    daysDiff = -1
    daysCount = 0
    daysNeed = 4
    startDate = None
    for s in cursor:
        diff = (today - s["date"]).days
        if diff > daysDiff:
            daysDiff = diff
            daysCount = daysCount + 1
        if daysCount > daysNeed:
            break
        if daysCount < daysNeed:
            continue

        startDate = s["date"]
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": [s["strategy"]]
            }
        else:
            stocks[code]["strategies"].append(s["strategy"])

    found = []
    for code, stock in stocks.iteritems():
        res = []
        for s in client.Trade[code].find({"date": {"$gt": startDate}}):
            res.append(s)
        if len(res) < daysNeed - 1:
            continue
        buyPrice = res[0]["open"]
        sellPrice1 = res[1]["high"]
        sellPrice2 = res[2]["high"]
        if buyPrice == 0:
            continue
        change1 = (sellPrice1 / buyPrice - 1) * 100
        change2 = (sellPrice2 / buyPrice - 1) * 100
        found.append({
            "code": code,
            "name": stock["name"],
            "strategies": stock["strategies"],
            "change1": change1,
            "change2": change2,
            "buy": buyPrice,
            "sell1": sellPrice1,
            "sell2": sellPrice2
        })

    found = sorted(found, key=lambda item: item["change2"], reverse=True)
    print pandas.DataFrame(found, columns=["code", "name", "change1", "change2", "buy", "sell1", "sell2", "strategies"])
    
Callback()

In [155]:
Strategy_1()

,code,name,change_0,change_-1,change_-2
0,600476,湘邮科技,10.029326,-9.978881,9.988386
1,300076,GQY视讯,10.085470,9.962406,1.720841
2,002795,永和智控,9.993869,9.979771,10.014837
3,300519,新光药业,10.004312,10.009488,10.020877
4,002806,华锋股份,10.006046,10.009977,0.000000
5,300526,中潜股份,10.010320,9.988649,9.987516
6,002863,今飞凯达,9.973522,10.000000,-42.296919
7,300690,双一科技,10.003206,10.017637,-4.319946
8,300693,盛弘股份,9.994223,10.009533,-5.721989
